## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score ,confusion_matrix
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier

In [2]:
train_data = pd.read_csv("C:/Users/loe_lin/Desktop/Machine learning/Day048/train.csv",header = None)
test_data = pd.read_csv("C:/Users/loe_lin/Desktop/Machine learning/Day048/test.csv",header = None)
trainLabels = pd.read_csv("C:/Users/loe_lin/Desktop/Machine learning/Day048/trainLabels.csv",header = None)

train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.850465,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.022959,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,0.513906,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332


In [3]:

x_train,x_test,y_train,y_test = train_test_split(train_data,trainLabels,test_size = 0.33,random_state = 42)

# print(metrics.mean_squared_error(y_test,y_pred))

In [4]:
TGB = XGBClassifier()
TGB.fit(x_train,y_train.values.ravel())
y_pred = TGB.predict(x_test)

In [5]:
print('acc',accuracy_score(y_pred,y_test))
# print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

acc 0.8666666666666667


In [6]:
n_estimators = [100,200,300,500]
max_depth = [5,10,20]
param_grid = dict(n_estimators = n_estimators,max_depth = max_depth)

grid_search = GridSearchCV(TGB,param_grid,scoring = "neg_mean_squared_error",n_jobs=-1, verbose=1)

grid_result = grid_search.fit(x_train,y_train)

print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# WGB_bestparam = GradientBoostingRegressor(max_depth = grid_result.best_params_["max_depth"],
#                                              n_estimators = grid_result.best_params_["n_estimators"])

# WGB_bestparam.fit(x_train,y_train)

# y_pred = WGB_bestparam.predict(x_test)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


C:\Users\loe_lin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


Best Accuracy: -0.129851 using {'max_depth': 5, 'n_estimators': 100}


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    4.6s finished
C:\Users\loe_lin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\loe_lin\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
TGB_bestparam = XGBClassifier(max_depth = grid_result.best_params_["max_depth"],
                                n_estimators = grid_result.best_params_["n_estimators"])

TGB_bestparam.fit(x_train,y_train)

y_pred = TGB_bestparam.predict(x_test)

print('acc',accuracy_score(y_pred,y_test))

acc 0.8757575757575757


In [8]:
TGB_best_pred = pd.DataFrame(y_pred)

TGB_best_pred.index += 1

TGB_best_pred.columns = ['Solution']
TGB_best_pred['Id'] = np.arange(1,TGB_best_pred.shape[0]+1)
TGB_best_pred = TGB_best_pred[['Id', 'Solution']]

TGB_best_pred.to_csv('Day048_model.csv',index=False)